# Ejercicios Scraping I

In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import re

## 1) Titulos de noticias
Descargar la pagina principal de noticias de Tecnología. Utilizar Regex para obtener los titulos de las noticias.

Ayuda: 
- los titulos estan encerrados por el tag `h2`
- No tiene que contenter el nombre de otra categoria

In [ ]:
infobae_url = 'https://www.infobae.com/tag/tecnologia/'

# Solución
response = requests.get(infobae_url).text

patron = r'<h2.*?>([^<>]+)</h2>'

titulos = re.findall(patron, response)

print(f'En total son {len(titulos)} titulos:\n{titulos}')


En total son 30:
['La jefa de ciberseguridad británica alertó sobre la amenaza de China a Occidente: “El desafío que define una época”', 'Los dilemas de la inteligencia artificial', 'Cuáles son los nuevos empleos que genera la Inteligencia Artificial y qué precauciones se deben tomar', '¿Cómo descargar el Microsoft Office gratis y legal en 2023?', 'Tendencias culturales que inspiran la forma de enseñar', 'La tecnología como solución de problemas', 'Quiénes son las 10 mentes brillantes de la ciencia y la tecnología elegidas por la revista TIME', 'Mujeres en tecnología: la oportunidad para reducir la brecha de género', 'La oficina del futuro: cómo serán los lugares de trabajo en el 2050', 'Así invierte su dinero Jeff Bezos, fundador de Amazon y uno de los hombres más ricos del mundo', 'Cómo funcionan los dispositivos digitales para el tratamiento médico de los niños con talla baja', 'Caja de compensación lanzó 500 becas para formación tecnológica e inglés en Bogotá y Cundinamarca', 'La t

## 2) Spinetta

Descargar letras de canciones

Utilizando beautiful soup descargar todas las canciones de [Spinetta](https://es.wikipedia.org/wiki/Luis_Alberto_Spinetta) que hay en [letras.com](https://www.letras.com/spinetta/)

In [ ]:
letras_url = "https://www.letras.com"

def descargar_letras(artista):
    # Solución
    dir_path = '/content/letras'
    path = os.path.join(dir_path, artista)

    if not os.path.exists(dir_path):
      os.mkdir(dir_path)
    if not os.path.exists(path):
      
      os.mkdir(path)

    os.chdir(path)

    html = requests.get(letras_url + '/' + artista).text
    soup = BeautifulSoup(html, 'html.parser')
    canciones = soup.find_all('a', class_ = 'song-name')

    # Se realizo un slice a la lista de canciones para no generar demasiados archivos
    # El objetivo es comprobar que funciona y descarga las letras de las 5 primeras canciones
    for cancion in canciones[:5]:
      titulo = cancion.span.text
      url = letras_url + cancion['href']
      song_html = requests.get(url).text
      song_soup = BeautifulSoup(song_html)

      letra_html = song_soup.find('div', attrs={'class': 'cnt-letra'})
      letra = ''

      for parrafo in letra_html.find_all('p'):
        texto = parrafo.get_text()
        # Agrega un salto de linea por cada letra mayuscula
        letra += re.sub(r'([A-Z])', '\n\g<1>', texto)
        # Agrega un salto de linea entre parrafos
        letra += '\n'

      with open(f'{titulo}.txt', 'w') as file:
        file.write(letra)
      
    os.chdir('/content')

    return 
        
artista = "luis-alberto-spinetta"
descargar_letras(artista)

## 3) Datos de la Formula 1

### 3.1) Obtener todos los campeones del mundo
Utilizando beautiful soup se pide descargar los siguientes datos:
- Año del campeonato
- Nombre del piloto
- Nombre del equipo
- Cantidad de carreras
- Cantidad de pilotos

In [ ]:
f1_piloto_url = 'https://pitwall.app/seasons?view=grid'

# Solución
pilotos = []
response = requests.get(f1_piloto_url)
soup = BeautifulSoup(response.text, 'html.parser')

grid_data = soup.find('div', id='seasons-grid')
for index, element in enumerate(grid_data.find_all('a')):
  
  anio = element.find('div', class_='top-label driver-number').text

  info = element.find('div', class_= 'info')
  piloto = info.find('h6').text
  try:
    equipo = info.find('p').text.strip()
  except AttributeError:
    print(f'No se pudo obtener el equipo campeon de {anio.strip()}')
    equipo = None

  labels = element.find('div', {'class': 'labels'}).text
  carreras = re.findall(r'(\d{1,3}) races', labels)[0]
  cant_pilotos = re.findall(r'(\d{1,3}) drivers', labels)[0]

  data = {
    'año': int(anio.strip()),
    'piloto': piloto.strip(),
    'equipo': equipo,
    'carreras': int(carreras),
    'total_pilotos': int(cant_pilotos)
  }
  pilotos.append(data)
    
# Con el objetivo de visualizar la informacion se utiliza pandas
import pandas as pd
df = pd.DataFrame(pilotos)
df

No se pudo obtener el equipo campeon de 1957
No se pudo obtener el equipo campeon de 1956
No se pudo obtener el equipo campeon de 1955
No se pudo obtener el equipo campeon de 1954
No se pudo obtener el equipo campeon de 1953
No se pudo obtener el equipo campeon de 1952
No se pudo obtener el equipo campeon de 1951
No se pudo obtener el equipo campeon de 1950


,año,piloto,equipo,carreras,total_pilotos
0,2023,Max Verstappen,Red Bull,23,20
1,2022,Max Verstappen,Red Bull,22,22
2,2021,Max Verstappen,Mercedes,22,21
3,2020,Lewis Hamilton,Mercedes,17,23
4,2019,Lewis Hamilton,Mercedes,21,20
...,...,...,...,...,...
69,1954,Juan Fangio,None,9,97
70,1953,Alberto Ascari,None,9,108
71,1952,Alberto Ascari,None,8,105
72,1951,Juan Fangio,None,8,84


### 3.2) Circuitos historicos
Utilizando beautiful soup se pide descargar los siguientes datos:
- Nombre del circuito
- Ubicacion
- Cantidad de carreras
- Link a la pagina del circuito

Ingresando a la pagina del circuito se debe obtener:
- El piloto con más victorias
- Cantidad de victorias
- El piloto con más Pole positions (piloto mas rapido en la clasificación)
- Cantidad de Pole positions

Nota: es recomendable crear una funcion para scrapear la pagina del circuito y devolver los valores pedidos

In [ ]:
f1_circuito_url = 'https://pitwall.app/circuits'

# Solución
def scrap_row(row):
  registro = row.find('tbody').find('tr')
  nombre = registro.find('td', class_='title').text
  cantidad = registro.find('td', class_='center title').text

  return nombre, cantidad


def scrap_circuito(url):
  base_url = 'https://pitwall.app'
  response_circ = requests.get(base_url + url)
  soup_circ = BeautifulSoup(response_circ.text, 'html.parser')
  
  tabla_vict = soup_circ.find_all('div', class_='most-wins')

  try:
    mas_victorias, cant_victorias = scrap_row(tabla_vict[0])
    mas_victorias = mas_victorias.strip()
    cant_victorias = int(cant_victorias)
  except AttributeError:
    print('No se pudieron obtener las victorias.')
    mas_victorias, cant_victorias = None, None

  try:
    mas_pole, cant_pole = scrap_row(tabla_vict[1])
    mas_pole = mas_pole.strip()
    cant_pole = int(cant_pole)
  except AttributeError:
    print('No se pudieron obtener las poles.')
    mas_pole, cant_pole = None, None

  return {
    'piloto_vict': mas_victorias,
    'cant_vict': cant_victorias,
    'piloto_pole': mas_pole,
    'cant_pole': cant_pole,
  }

pistas = []
response = requests.get(f1_circuito_url)
soup = BeautifulSoup(response.text, 'html.parser')

tabla = soup.find('tbody')

for circuito in tabla.find_all('tr'):
  
  a_tag = circuito.find('a')
  url = a_tag['href']
  nombre = a_tag.text

  ubicacion = circuito.find('td', {'class': 'nowrap'}).text
  carreras = circuito.find('td', {'class': 'minlg center'}).text

  data_circuito = scrap_circuito(url)

  data = {
      'nombre': nombre,
      # replace soluciona un problema con el caracter en el HTML
      'ubicacion': ubicacion.strip().replace('\xa0', ' '),
      'carreras': int(carreras),
      **data_circuito
  }
  pistas.append(data)

import pandas as pd

df = pd.DataFrame(pistas)
df

No se pudieron obtener las victorias.
No se pudieron obtener las poles.
No se pudieron obtener las victorias.
No se pudieron obtener las poles.


,nombre,ubicacion,carreras,piloto_vict,cant_vict,piloto_pole,cant_pole
0,A1-Ring,"Spielburg, Austria",25,Alain Prost,3.0,Niki Lauda,3.0
1,Adelaide Street Circuit,"Adelaide, Australia",11,Alain Prost,2.0,Ayrton Senna,6.0
2,Ain Diab,"Casablanca, Morocco",1,Stirling Moss,1.0,Mike Hawthorn,1.0
3,Aintree,"Liverpool, UK",5,Stirling Moss,2.0,Stirling Moss,2.0
4,Albert Park Grand Prix Circuit,"Melbourne, Australia",26,Michael Schumacher,4.0,Lewis Hamilton,8.0
...,...,...,...,...,...,...,...
74,Valencia Street Circuit,"Valencia, Spain",5,Sebastian Vettel,2.0,Sebastian Vettel,3.0
75,Watkins Glen,"New York State, USA",20,Jim Clark,3.0,Graham Hill,3.0
76,Yas Marina Circuit,"Abu Dhabi, UAE",15,Lewis Hamilton,5.0,Lewis Hamilton,5.0
77,Zeltweg,"Styria, Austria",1,Lorenzo Bandini,1.0,Graham Hill,1.0
